In [1]:
# using GPT to extract the keyword

In [49]:
import openai
from secret import *
openai.api_key = OPENAI_API_KEY

from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
    retry_if_exception_type
)  # for exponential backoff

@retry(
    retry=retry_if_exception_type((openai.error.APIError, openai.error.APIConnectionError, openai.error.RateLimitError, openai.error.ServiceUnavailableError, openai.error.Timeout)), 
    wait=wait_random_exponential(multiplier=1, max=60), 
    stop=stop_after_attempt(10)
)
def chat_completion_with_backoff(**kwargs):
    return openai.ChatCompletion.create(**kwargs)

In [96]:
response = "The relevant clinical characteristics for the etiological diagnosis of Atelectasis include Cyanosis and Increased Heart Rate."
# prompt = f"""For the subsequent paragr|aph, isolate solely those clinical keywords that are can be represented as numerical values or boolean values: \"{response}\""""
# prompt = f"""For the subsequent paragraph, isolate solely those clinical keywords: \"{response}\""""
prompt = f"""For the subsequent paragraph, isolate solely those clinical keywords that are can be represented as symptoms or numerical values or boolean values: \"{response}\""""

In [97]:
res = chat_completion_with_backoff(
                # model="gpt-3.5-turbo",
                model="gpt-4",
                messages=[
                    {"role": "system", "content": "You are a helpful clinical expert."},
                    # {"role": "system", "content": "You are an experienced radiologist. Use a keyword-based report to answer the following questions."},
                    {"role": "user", "content": prompt},
                ],
                temperature=0,
                n=1,
            )

In [98]:
keywords = [k.lower().strip() for k in res['choices'][0]['message']['content'].replace(".", "").split(",") if k.lower().strip() != "atelectasis"]

In [99]:
keywords

['cyanosis', 'increased heart rate']

In [ ]:
# adding features, then 